### Testing: Data Cleaning and Quality Control for Pregnancy Deep Phenotyping Metabolomics Data
### Piekos Lab, Kayla Xu
### 01/26/2026

In [2]:
# environment
import pandas as pd 
import numpy as np 


In [ ]:
# Metabolomics Files:
### 050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx
### 050725_Sadovsky DP3 Plasma Polar Untargeted_ALL copy.xlsx
placentaPos_raw1 = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",skiprows=2)
plasmaPos_raw1 = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Plasma Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",skiprows=2)

In [3]:
# extract "Area" information
placentaPos = placentaPos_raw
not_unnamed =["Unnamed" not in s for s in placentaPos.columns]
sample_area = placentaPos.iloc[0:1, not_unnamed]
sample_area = sample_area.set_index("Sample ID").transpose()
sample_area.columns = ["Area"]
sample_area.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_sample_area.csv")

In [4]:
# Extract compound metadata
not_sample =["DP3" not in s and "Pooled" not in s for s in placentaPos.columns]
placentaPos_comps = placentaPos.iloc[:, not_sample]
placentaPos_comps = placentaPos_comps.rename(columns=placentaPos_comps.iloc[0,:]).drop([0])
placentaPos_comps.iloc[0,0] = "p01" # D3-alanine-ISTD
placentaPos_comps.iloc[1,0] = "p02" # 13C1-creatinine_ISTD
placentaPos_comps.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_compounds.csv")


In [5]:
# Extract expression data
is_expression = ["DP3" in s or "Pooled" in s for s in placentaPos.columns]
expression = placentaPos.iloc[:, is_expression]
expression = expression.drop([0])
expression.index = placentaPos_comps["Export Order"]
expression = expression.transpose()
expression.to_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_expression.csv")

In [6]:
expression.index

Index(['Pooled Cntrl', 'Pooled Cntrl.1', 'Pooled Cntrl.2', 'Pooled Cntrl.3',
       'Pooled Cntrl.4', 'Pooled Cntrl.5', 'Pooled Cntrl.6', 'Pooled Cntrl.7',
       'Pooled Cntrl.8', 'Pooled Cntrl.9',
       ...
       'DP3-0387', 'DP3-0389', 'DP3-0398E', 'DP3-0399', 'DP3-0404', 'DP3-0409',
       'DP3-0416', 'DP3-0419', 'DP3-0420', 'DP3-0423E'],
      dtype='str', length=137)

In [7]:
def get_batch(df):
    is_sample = df.columns.notna()
    temp = df.iloc[0:1,is_sample]
    temp = temp.set_index("Sample ID").transpose()
    temp.columns = ["batch"]
    temp["batch"] = [s.split(": ")[1].split("_")[0] for s in temp["batch"]]
    temp.index = temp.index.rename("Sample_ID")
    return temp




In [8]:
# extract compound metadata
def get_compounds(df):
    not_sample = df.columns.isna()
    temp = df.iloc[:, not_sample]
    temp.columns = temp.iloc[0,:]
    temp = temp.drop(temp.index[0])
    temp.index = temp.index.rename("Export Order")
    return temp


In [9]:

# extract expression data
def get_expression(df, comp):
    is_sample = df.columns.notna()
    temp = df.iloc[:, is_sample].drop(columns="Sample ID").drop(df.index[0])
    temp.index = comp["Name"].fillna(comp.index.to_series())
    temp = temp.transpose()
    temp.index = temp.index.rename("compound")
    return temp



In [10]:
# call all csv generating function
def generate_files(df, file_output, e):
    get_batch(df).to_csv(file_output + "/" + e +"_batch.csv")
    comp = get_compounds(df)
    comp.to_csv(file_output + "/" + e + "_compounds.csv")
    get_expression(df, comp).to_csv(file_output + "/" + e + "_expression.csv")

def clean_df(df):
    df.columns = df.iloc[0,:]
    df = df.iloc[1:,:]
    df.iloc[1,0] = "01" # two moleclues not in the export order
    df.iloc[2,0] = "02" # slightly different for each file
    df.index = df.iloc[:,0]
    df = df.iloc[:, 1:]
    return df

# helper function
def extract_data(file_input, file_output):
    #file = open(file_input, mode="r")
    file = file_input
    file_pos = pd.read_excel(file, sheet_name="POS Compounds",header=None).dropna(how='all')
    file_neg = pd.read_excel(file, sheet_name="NEG Compounds",header=None).dropna(how='all')
  
    # remove empty rows and set index/columns
    file_pos = clean_df(file_pos)
    file_neg = clean_df(file_neg)

    #generate files
    generate_files(file_pos, file_output, "pos")
    generate_files(file_neg, file_output, "neg")



In [11]:
extract_data('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', '/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta')

In [12]:

placentaPos_raw = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",header=None).dropna(how='all')
placentaPos_raw.columns = placentaPos_raw.iloc[0,:]
placentaPos_raw = placentaPos_raw.iloc[1:,:]
placentaPos_raw.iloc[1,0] = "01" # two moleclues not in the export order
placentaPos_raw.iloc[2,0] = "02" # slightly different for each file
placentaPos_raw.index = placentaPos_raw.iloc[:,0]
placentaPos_raw = placentaPos_raw.iloc[:, 1:]
placentaPos_raw

2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DP3-0398E,DP3-0399,DP3-0404,DP3-0409,DP3-0416,DP3-0419,DP3-0420,DP3-0423E,NaN,NaN
nan,,,,,,,,,,,,,,,,,,,,,
Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,Area: 032425_Sadovsky_Polar_Pos_45.raw (F115),Area: 032425_Sadovsky_Polar_Pos_46.raw (F116),Area: 032425_Sadovsky_Polar_Pos_47.raw (F117),Area: 032425_Sadovsky_Polar_Pos_48.raw (F118),Area: 032425_Sadovsky_Polar_Pos_49.raw (F119),Area: 032425_Sadovsky_Polar_Pos_50.raw (F120),Area: 032425_Sadovsky_Polar_Pos_51.raw (F121),Area: 032425_Sadovsky_Polar_Pos_52.raw (F122),Peak Rating (Max.),Peak Rating QC (Max.)
01,D3-alanine-ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,401151.873605,373625.231405,364593.320492,399287.829141,435095.699504,272278.020734,436893.81994,354308.6104,NaN,NaN
02,13C1-creatinine_ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22865033.147103,23508151.282216,23660342.335612,18189295.140684,22990171.893712,18648420.254607,22036274.984124,19718638.033679,NaN,NaN
p1,NaN,C4 H4 O7 P2,Full match,No results,No results,No results,No match,No match,4.39,225.94422,...,1299977851.71867,1319932558.19584,1415801523.75981,1354812550.03793,1246002605.39726,1280463349.87845,1365353348.293,1331074984.32775,9.2,5.5
p2,Phosphate,H3 O4 P,No results,No results,No results,Full match,Full match,Full match,-0.08,97.97689,...,284659338.91841,224398236.582277,260621107.991995,152190993.760215,144181458.718533,259967809.838309,118226509.757648,297492629.450178,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,NaN,C26 H38 N2 O11 S,Full match,No results,No results,No results,No match,No match,3.47,586.22167,...,38495.799017,59486.156988,43756.364913,30018.671239,23856.126696,50097.705991,50517.907992,37939.473894,5.9,5.8
p3701,"1,2,4-Trimethylbenzene",C9 H12,Full match,No results,No results,No results,Partial match,Full match,-0.79,120.09381,...,7232.991172,6344.848361,4051.1737,3779.421277,5112.78309,5257.716925,5248.152774,34064.236283,6.4,6.4
p3702,NaN,C12 H5 N4 O P,Full match,No results,No results,No results,No results,No results,1.15,252.02039,...,1809.39338,28695.953183,6014.48284,112197.881939,1738.518821,7674.30238,1616.330299,37309.011028,5.2,3.2


In [13]:
df = placentaPos_raw
is_sample = df.columns.notna()
temp = df.iloc[0:1,is_sample]
temp = temp.set_index("Sample ID").transpose()
temp.columns = ["batch"]
temp["batch"] = [s.split(": ")[1].split("_")[0] for s in temp["batch"]]
temp
#test

,batch
2,
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
...,...
DP3-0409,032425
DP3-0416,032425
DP3-0419,032425


In [14]:
df = placentaPos_raw
not_sample = df.columns.isna()
temp = df.iloc[:, not_sample]
temp.columns = temp.iloc[0,:]
#temp = temp.drop(temp.index[0])
#temp.index[0] = "01" # two moleclues not in the export order
#temp.index[1] = "02" # slightly different for each file
temp

Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,# Usable QC,RSD QC Areas [%],RSD Corr. QC Areas [%],Mass List Match: HMDB All metabolites (v5. 217719 cpds),Mass List Match: PFAS_NIST,Mass List Match: Natural Products Atlas 2023_06,Mass List Match: Polar ISTD,MS2,Peak Rating (Max.),Peak Rating QC (Max.)
nan,,,,,,,,,,,,,,,,,,,,,
Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,# Usable QC,RSD QC Areas [%],RSD Corr. QC Areas [%],Mass List Match: HMDB All metabolites (v5. 217...,Mass List Match: PFAS_NIST,Mass List Match: Natural Products Atlas 2023_06,Mass List Match: Polar ISTD,MS2,Peak Rating (Max.),Peak Rating QC (Max.)
01,D3-alanine-ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02,13C1-creatinine_ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p1,NaN,C4 H4 O7 P2,Full match,No results,No results,No results,No match,No match,4.39,225.94422,...,13,5,1,Single match found,No matches found,No matches found,No matches found,DDA for preferred ion,9.2,5.5
p2,Phosphate,H3 O4 P,No results,No results,No results,Full match,Full match,Full match,-0.08,97.97689,...,13,22,7,Multiple matches found,No matches found,No matches found,No matches found,DDA for other ion,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,NaN,C26 H38 N2 O11 S,Full match,No results,No results,No results,No match,No match,3.47,586.22167,...,13,14,3,No matches found,No matches found,Single match found,No matches found,DDA for preferred ion,5.9,5.8
p3701,"1,2,4-Trimethylbenzene",C9 H12,Full match,No results,No results,No results,Partial match,Full match,-0.79,120.09381,...,8,21,7,Multiple matches found,No matches found,Single match found,No matches found,No MS2,6.4,6.4
p3702,NaN,C12 H5 N4 O P,Full match,No results,No results,No results,No results,No results,1.15,252.02039,...,8,24,6,No matches found,No matches found,No matches found,No matches found,No MS2,5.2,3.2


In [22]:
placentaPos_raw1 = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/050725_Sadovsky DP3 Placenta Polar Untargeted_ALL copy.xlsx', sheet_name="POS Compounds",header=None).dropna(how='all')
#df = clean_df(placentaPos_raw)
#df

In [16]:
b =get_batch(df)
b

,batch
Sample_ID,
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
Pooled Cntrl,062323
...,...
DP3-0409,032425
DP3-0416,032425
DP3-0419,032425


In [17]:
t = get_compounds(df)
t

Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,# Usable QC,RSD QC Areas [%],RSD Corr. QC Areas [%],Mass List Match: HMDB All metabolites (v5. 217719 cpds),Mass List Match: PFAS_NIST,Mass List Match: Natural Products Atlas 2023_06,Mass List Match: Polar ISTD,MS2,Peak Rating (Max.),Peak Rating QC (Max.)
Export Order,,,,,,,,,,,,,,,,,,,,,
01,D3-alanine-ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02,13C1-creatinine_ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p1,NaN,C4 H4 O7 P2,Full match,No results,No results,No results,No match,No match,4.39,225.94422,...,13,5,1,Single match found,No matches found,No matches found,No matches found,DDA for preferred ion,9.2,5.5
p2,Phosphate,H3 O4 P,No results,No results,No results,Full match,Full match,Full match,-0.08,97.97689,...,13,22,7,Multiple matches found,No matches found,No matches found,No matches found,DDA for other ion,8,7
p3,Acetyl-L-carnitine,C9 H17 N O4,Full match,Full match,No results,No results,Full match,Full match,-0.18,203.11572,...,13,8,2,Multiple matches found,No matches found,No matches found,No matches found,DDA for preferred ion,8.6,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,NaN,C26 H38 N2 O11 S,Full match,No results,No results,No results,No match,No match,3.47,586.22167,...,13,14,3,No matches found,No matches found,Single match found,No matches found,DDA for preferred ion,5.9,5.8
p3701,"1,2,4-Trimethylbenzene",C9 H12,Full match,No results,No results,No results,Partial match,Full match,-0.79,120.09381,...,8,21,7,Multiple matches found,No matches found,Single match found,No matches found,No MS2,6.4,6.4
p3702,NaN,C12 H5 N4 O P,Full match,No results,No results,No results,No results,No results,1.15,252.02039,...,8,24,6,No matches found,No matches found,No matches found,No matches found,No MS2,5.2,3.2


In [18]:
x = get_expression(df, t.index)
x

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
is_sample = df.columns.notna()
temp = df.iloc[:, is_sample].drop(columns="Sample ID")
#temp.index = ids
#temp = temp.transpose()
#temp.index = temp.index.rename("Sample_ID")
temp


2,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,Pooled Cntrl,...,DP3-0387,DP3-0389,DP3-0398E,DP3-0399,DP3-0404,DP3-0409,DP3-0416,DP3-0419,DP3-0420,DP3-0423E
nan,,,,,,,,,,,,,,,,,,,,,
Export Order,Area: 062323_Sadovsky_placenta_pospool1.raw (F65),Area: 062323_Sadovsky_placenta_pospool2.raw (F66),Area: 062323_Sadovsky_placenta_pospool3.raw (F67),Area: 062323_Sadovsky_placenta_pospool4.raw (F68),Area: 062323_Sadovsky_placenta_pospool5.raw (F69),Area: 062323_Sadovsky_placenta_pospool6.raw (F70),Area: 032425_Sadovsky_Polar_Pos_Pool1.raw (F134),Area: 032425_Sadovsky_Polar_Pos_Pool2.raw (F135),Area: 032425_Sadovsky_Polar_Pos_Pool3.raw (F136),Area: 032425_Sadovsky_Polar_Pos_Pool4.raw (F137),...,Area: 032425_Sadovsky_Polar_Pos_43.raw (F113),Area: 032425_Sadovsky_Polar_Pos_44.raw (F114),Area: 032425_Sadovsky_Polar_Pos_45.raw (F115),Area: 032425_Sadovsky_Polar_Pos_46.raw (F116),Area: 032425_Sadovsky_Polar_Pos_47.raw (F117),Area: 032425_Sadovsky_Polar_Pos_48.raw (F118),Area: 032425_Sadovsky_Polar_Pos_49.raw (F119),Area: 032425_Sadovsky_Polar_Pos_50.raw (F120),Area: 032425_Sadovsky_Polar_Pos_51.raw (F121),Area: 032425_Sadovsky_Polar_Pos_52.raw (F122)
01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,454705.358703,436747.959268,401151.873605,373625.231405,364593.320492,399287.829141,435095.699504,272278.020734,436893.81994,354308.6104
02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19094360.053184,27696422.442596,22865033.147103,23508151.282216,23660342.335612,18189295.140684,22990171.893712,18648420.254607,22036274.984124,19718638.033679
p1,1529697732.10807,1368644592.09096,1529095145.48298,1594294947.22906,1558061941.6777,1310787636.26796,1328624426.10309,1344193102.16676,1364611344.9006,1286360927.71433,...,1400241997.33195,1415624952.17838,1299977851.71867,1319932558.19584,1415801523.75981,1354812550.03793,1246002605.39726,1280463349.87845,1365353348.293,1331074984.32775
p2,960934888.044903,1017308011.36853,1024927927.19107,1061045171.20838,1075289478.05843,1139913107.67262,124139614.511546,227930546.262139,88045281.776889,248049614.116663,...,155063633.967029,218510011.336255,284659338.91841,224398236.582277,260621107.991995,152190993.760215,144181458.718533,259967809.838309,118226509.757648,297492629.450178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,39079.646816,37595.24349,41865.668458,27640.514842,23745.862385,28983.02243,49275.668837,45309.560097,42586.358396,48846.685818,...,58239.195285,51006.320238,38495.799017,59486.156988,43756.364913,30018.671239,23856.126696,50097.705991,50517.907992,37939.473894
p3701,32186.048274,14915.13476,30473.225647,34969.834441,23107.849268,37536.088718,7480.893067,3667.167423,3550.747425,3968.712862,...,9320.302387,12289.337498,7232.991172,6344.848361,4051.1737,3779.421277,5112.78309,5257.716925,5248.152774,34064.236283
p3702,1601.922548,1483.575237,1565.337035,2063.441685,9204.205117,1842.760244,4239.181509,4944.474686,3977.297126,12219.016851,...,2253.014273,1309.799457,1809.39338,28695.953183,6014.48284,112197.881939,1738.518821,7674.30238,1616.330299,37309.011028


In [ ]:
t

Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,# Usable QC,RSD QC Areas [%],RSD Corr. QC Areas [%],Mass List Match: HMDB All metabolites (v5. 217719 cpds),Mass List Match: PFAS_NIST,Mass List Match: Natural Products Atlas 2023_06,Mass List Match: Polar ISTD,MS2,Peak Rating (Max.),Peak Rating QC (Max.)
Export Order,,,,,,,,,,,,,,,,,,,,,
01,D3-alanine-ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02,13C1-creatinine_ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p1,NaN,C4 H4 O7 P2,Full match,No results,No results,No results,No match,No match,4.39,225.94422,...,13,5,1,Single match found,No matches found,No matches found,No matches found,DDA for preferred ion,9.2,5.5
p2,Phosphate,H3 O4 P,No results,No results,No results,Full match,Full match,Full match,-0.08,97.97689,...,13,22,7,Multiple matches found,No matches found,No matches found,No matches found,DDA for other ion,8,7
p3,Acetyl-L-carnitine,C9 H17 N O4,Full match,Full match,No results,No results,Full match,Full match,-0.18,203.11572,...,13,8,2,Multiple matches found,No matches found,No matches found,No matches found,DDA for preferred ion,8.6,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,NaN,C26 H38 N2 O11 S,Full match,No results,No results,No results,No match,No match,3.47,586.22167,...,13,14,3,No matches found,No matches found,Single match found,No matches found,DDA for preferred ion,5.9,5.8
p3701,"1,2,4-Trimethylbenzene",C9 H12,Full match,No results,No results,No results,Partial match,Full match,-0.79,120.09381,...,8,21,7,Multiple matches found,No matches found,Single match found,No matches found,No MS2,6.4,6.4
p3702,NaN,C12 H5 N4 O P,Full match,No results,No results,No results,No results,No results,1.15,252.02039,...,8,24,6,No matches found,No matches found,No matches found,No matches found,No MS2,5.2,3.2


In [ ]:
t["Name"].fillna(t.index.to_series())

Export Order
01              D3-alanine-ISTD
02         13C1-creatinine_ISTD
p1                           p1
p2                    Phosphate
p3           Acetyl-L-carnitine
                  ...          
p3700                     p3700
p3701    1,2,4-Trimethylbenzene
p3702                     p3702
p3703                     p3703
p3704                Zonisamide
Name: Name, Length: 3706, dtype: str

In [17]:
# lipidomic files

placentaPos_raw = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/060525_Sadovsky Placenta Lipids Untargeted_ALL.xlsx', sheet_name="POS Lipids", header=None).dropna(how='all').iloc[1:,:]
plasmaPos_raw = pd.read_excel('/Users/kaylaxu/Desktop/data/raw_data/072925 Sadovsky Plasma Lipids Untargeted ALL.xlsx', sheet_name="Plasma POS Lipids", header=None).dropna(how='all').iloc[1:,:]

In [20]:
def clean_df(df):
    df.columns = df.iloc[0,:]
    df = df.iloc[1:,:]
    i = 1
    while pd.isna(df.iloc[i,0]):
        print(i)
        df.iloc[i,0] = "c" + str(i) # placeholder names for experimental controls
        i += 1
    df.index = df.iloc[:,0]
    df = df.iloc[:, 1:]
    return df

In [21]:
clean_df(placentaPos_raw)

1
2
3
4
5
6


2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sample ID,...,DP3-0184E,DP3-0185,DP3-0200,DP3-0202,DP3-0204,DP3-0207,DP3-0214,DP3-0216,DP3-0233E,DP3-0265
nan,,,,,,,,,,,,,,,,,,,,,
Export Order,LipidID,LipidGroup,Charge,CalcMz,BaseRt,Class,SubClass,AdductIon,IonFormula,MolStructure,...,062923_Sadovsky_placenta_lipid_pos53',062923_Sadovsky_placenta_lipid_pos54',062923_Sadovsky_placenta_lipid_pos55',062923_Sadovsky_placenta_lipid_pos56',062923_Sadovsky_placenta_lipid_pos57',062923_Sadovsky_placenta_lipid_pos58',062923_Sadovsky_placenta_lipid_pos59',062923_Sadovsky_placenta_lipid_pos60',062923_Sadovsky_placenta_lipid_pos61',062923_Sadovsky_placenta_lipid_pos62'
c1,18:1_LPC-d7,POS ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,371204237.383897,374024578.807242,347642728.356562,393720326.06174,400154562.012124,392069167.156037,372768189.736998,378161248.606435,347637774.16379,442552940.056538
c2,18:1_SM-d9,POS ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,204187855.370871,219503947.02439,213494512.319301,212704255.965344,224759263.509306,218627540.773187,220579267.504861,232330039.366029,192334574.119849,253824355.106281
c3,15:0-18:1_PC-d7,POS ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,875398936.903653,910951935.9132,918070701.907078,925156528.174539,954893603.231055,931765627.385973,954035357.237838,980292436.009874,857239703.084934,1054654905.81318
c4,15:0-18:1_DG-d7_M+NH4,POS ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15956832.558493,15666120.43051,16602657.970268,18414758.314823,18522285.646132,16432862.713961,16627831.487552,18198796.513689,14005409.537925,19892463.93214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,Hex2Cer(d18:1_16:0)+H-H2O,Hex2Cer(d34:1)+H-H2O,1,844.614455,9.251325,Hex2Cer,Ceramide disaccharides,M+H-H2O,C46 H86 N1 O12,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)OC2C(C(C...,...,287.529073,520.603493,1620.512841,1399.151894,4648.487912,1333.505442,0,983.470499,319.439068,929.688073
3013,Hex2Cer(d18:1_16:0)+H-H2O,Hex2Cer(d34:1)+H-H2O,1,844.614455,9.270778,Hex2Cer,Ceramide disaccharides,M+H-H2O,C46 H86 N1 O12,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)OC2C(C(C...,...,287.529073,520.603493,1620.512841,1399.151894,4648.487912,1333.505442,0,983.470499,319.439068,929.688073
3180,Hex3Cer(d18:1_17:0)+H,Hex3Cer(d35:1)+H,1,1038.693495,9.860028,Hex3Cer,Ceramide N-acetyltrisaccharide,M+H,C53 H100 N1 O18,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)OC2C(C(C...,...,257.964439,1285.707055,946.977767,NP,949.985853,464.518513,460.384697,2057.928499,1124.977363,NP


In [24]:
clean_df(placentaPos_raw1)

1
2


2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DP3-0398E,DP3-0399,DP3-0404,DP3-0409,DP3-0416,DP3-0419,DP3-0420,DP3-0423E,NaN,NaN
nan,,,,,,,,,,,,,,,,,,,,,
Export Order,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: Metabolika Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [ppm],Calc. MW,...,Area: 032425_Sadovsky_Polar_Pos_45.raw (F115),Area: 032425_Sadovsky_Polar_Pos_46.raw (F116),Area: 032425_Sadovsky_Polar_Pos_47.raw (F117),Area: 032425_Sadovsky_Polar_Pos_48.raw (F118),Area: 032425_Sadovsky_Polar_Pos_49.raw (F119),Area: 032425_Sadovsky_Polar_Pos_50.raw (F120),Area: 032425_Sadovsky_Polar_Pos_51.raw (F121),Area: 032425_Sadovsky_Polar_Pos_52.raw (F122),Peak Rating (Max.),Peak Rating QC (Max.)
c1,D3-alanine-ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,401151.873605,373625.231405,364593.320492,399287.829141,435095.699504,272278.020734,436893.81994,354308.6104,NaN,NaN
c2,13C1-creatinine_ISTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22865033.147103,23508151.282216,23660342.335612,18189295.140684,22990171.893712,18648420.254607,22036274.984124,19718638.033679,NaN,NaN
p1,NaN,C4 H4 O7 P2,Full match,No results,No results,No results,No match,No match,4.39,225.94422,...,1299977851.71867,1319932558.19584,1415801523.75981,1354812550.03793,1246002605.39726,1280463349.87845,1365353348.293,1331074984.32775,9.2,5.5
p2,Phosphate,H3 O4 P,No results,No results,No results,Full match,Full match,Full match,-0.08,97.97689,...,284659338.91841,224398236.582277,260621107.991995,152190993.760215,144181458.718533,259967809.838309,118226509.757648,297492629.450178,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p3700,NaN,C26 H38 N2 O11 S,Full match,No results,No results,No results,No match,No match,3.47,586.22167,...,38495.799017,59486.156988,43756.364913,30018.671239,23856.126696,50097.705991,50517.907992,37939.473894,5.9,5.8
p3701,"1,2,4-Trimethylbenzene",C9 H12,Full match,No results,No results,No results,Partial match,Full match,-0.79,120.09381,...,7232.991172,6344.848361,4051.1737,3779.421277,5112.78309,5257.716925,5248.152774,34064.236283,6.4,6.4
p3702,NaN,C12 H5 N4 O P,Full match,No results,No results,No results,No results,No results,1.15,252.02039,...,1809.39338,28695.953183,6014.48284,112197.881939,1738.518821,7674.30238,1616.330299,37309.011028,5.2,3.2
